# Imports

In [2]:
import os
import requests
import shutil
from pathlib import Path
import random

In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data Loading